# U.S. Housing Price Trends — Interactive Visualizations

This notebook visualizes historical U.S. housing price data (1975–2024) through two complementary views:
1. **Animated Choropleth Map** — A color-coded U.S. map showing year-over-year price changes by state, animated across time.
2. **Bar Chart Race** — A ranked horizontal bar chart showing cumulative house price growth, animated to reveal how states overtake each other over decades.

In [ ]:
import plotly.express as px
import pandas as pd

## Animated Choropleth Map — Annual Price Change by State

### Load Data

The dataset contains annual percentage change in the FHFA House Price Index (HPI) for each state, along with a 3-year rolling average to smooth out short-term volatility.

In [ ]:
df = pd.read_csv("../output/state_growth_rates.csv")
df.head(10)

### Build Animated Choropleth

The choropleth uses a **Red–Yellow–Green** color scale (range: -20% to +20%) so that:
- **Red** = declining prices (negative annual change)
- **Yellow** = flat / modest growth
- **Green** = strong price appreciation

State abbreviation labels are overlaid at each state's geographic centroid. The animation frame slider at the bottom lets you scrub through years or press play to watch the full 1975–2024 sequence. Key patterns to look for:
- The **2006–2009 housing crash** turning most of the map red
- The **post-COVID boom (2020–2022)** lighting up nearly every state in green
- Regional differences — coastal states tend to show higher volatility than the Midwest

In [ ]:
import plotly.graph_objects as go

# Scrape state centroids from Google's public dataset
tables = pd.read_html('https://developers.google.com/public-data/docs/canonical/states_csv')
state_coords = tables[0]
state_coords.columns = ['state', 'latitude', 'longitude', 'name']

# Merge centroids into house price data
df = df.merge(state_coords[['state', 'latitude', 'longitude']], left_on='Abbreviation', right_on='state')

# Create animated choropleth with year slider
fig = px.choropleth(
    df,
    locations='Abbreviation',
    locationmode='USA-states',
    color='Annual Change (%)',
    color_continuous_scale='RdYlGn',
    range_color=(-20, 20),
    scope='usa',
    animation_frame='Year',
    labels={'Annual Change (%)': 'Annual Change (%)'},
    title='Annual House Price Change by State'
)

# State label trace (uses first year's data for coordinates)
coords = df.drop_duplicates(subset='Abbreviation')
label_trace = go.Scattergeo(
    locationmode='USA-states',
    lon=coords['longitude'],
    lat=coords['latitude'],
    text=coords['Abbreviation'],
    mode='text',
    textfont=dict(size=8, color='white'),
    showlegend=False,
    hoverinfo='skip'
)

# Add labels to the base figure
fig.add_trace(label_trace)

# Add labels to every animation frame so they persist
for frame in fig.frames:
    frame.data = (*frame.data, label_trace)

fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0})
fig.show()